In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
import numpy as np
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

In [2]:
geo_path = "data/state_data/geo/ppp_geo.csv" #data as of 11/21/21
ppp_geo= pd.read_csv(geo_path)
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,NaN
1,1,1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,NaN
2,2,2,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,NaN
3,3,3,3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,...,999 Other,2361 Residential Building Construction,2361.0,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,NaN
4,4,4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,NaN


In [3]:
ppp_geonull=ppp_geo.loc[ppp_geo["Lat"].isnull()] #457467
ppp_geonull.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
369,369,369,369,369,369,5048318601,3/20/21,1084,PPS,DELTA INDUSTRIAL SERVICES INC.,...,999 Other,5511 Management of Companies and Enterprises,5511.0,ALASKA DISTRICT OFFICE,"b $350,000-1 million",10 to 19,"1229 RICHARDSON HWY, DELTA JUNCTION, AK",NaN,NaN,NaN
428,428,428,428,428,428,5085327008,4/5/20,1084,PPP,"DELTA INDUSTRIAL SERVICES, INC.",...,999 Other,5511 Management of Companies and Enterprises,5511.0,ALASKA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1229 Richardson Hwy, DELTA JUNCTION, AK",NaN,NaN,NaN
1865,1865,1865,1865,1865,1865,4637628408,2/6/21,1084,PPS,MCKINLEY FIRE PROTECTION LLC,...,999 Other,2382 Building Equipment Contractors,2382.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1706 Ship Ave, Anchorage, AK",NaN,NaN,NaN
2274,2274,2274,2274,2274,2274,5337318401,2/8/21,1084,PPS,THE ALASKA CENTRE FOR CREATIVE RENEWAL LLC,...,999 Other,7211 Traveler Accommodation,7211.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"46877 STILLPOINT TRL, HALIBUT COVE, AK",NaN,NaN,NaN
2379,2379,2379,2379,2379,2379,9891427104,4/15/20,1084,PPP,MCKINLEY FIRE PROTECTION,...,999 Other,2382 Building Equipment Contractors,2382.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1706 SHIP AVENUE, ANCHORAGE, AK",NaN,NaN,NaN


In [4]:
ppp_geonull["BorrowerState"].value_counts()

TX    76183
NY    74112
FL    60040
MI    29806
WA    23754
CO    19741
MN    19730
WI    17354
LA    13668
OR    13456
UT     9295
IA     8295
MO     6946
NE     5804
HI     4980
NM     4485
ID     4416
ND     3222
MT     3144
PR     2952
SD     2695
OK     2535
IN     2348
WY     2139
SC     2045
MS     1589
NV     1569
KS     1485
GU      443
RI      414
VI      290
MP       82
NC       11
PA        8
AL        8
AK        5
AZ        4
TN        3
MA        3
NJ        3
ME        2
IL        2
AR        2
GA        1
CT        1
WV        1
OH        1
Name: BorrowerState, dtype: int64

In [5]:
ppp_togeoMO = ppp_geonull.loc[ppp_geonull["BorrowerState"] == "MO"]
ppp_togeoMO #16350

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
498556,498556,498556,498556,499870,499870,5467867110,4/13/20,721,PPP,RUSSELL CELLULAR INC,...,999 Other,5179 Other Telecommunications,5179.0,SPRINGFIELD BRANCH OFFICE,e $5-10 million,500 or more,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",NaN,NaN,NaN
498557,498557,498557,498557,499871,499871,5529817004,4/5/20,709,PPP,"SUPERIOR BOWEN ASPHALT CO., LLC",...,324 Petroleum & Coal Products,3241 Petroleum and Coal Products Manufacturing,3241.0,KANSAS CITY DISTRICT OFFICE,e $5-10 million,100 to 249,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",NaN,NaN,NaN
498558,498558,498558,498558,499872,499872,6105917109,4/14/20,709,PPP,PROENERGY ACQUISITION COMPANY,...,333 Machinery,"3336 Engine, Turbine, and Power Transmission E...",3336.0,KANSAS CITY DISTRICT OFFICE,e $5-10 million,500 or more,"2001 ProEnergy Blvd, SEDALIA, MO",NaN,NaN,NaN
498566,498566,498566,498566,499880,499880,8159007210,4/28/20,709,PPP,"JACK COOPER TRANSPORT COMPANY, LLC",...,999 Other,4841 General Freight Trucking,4841.0,KANSAS CITY DISTRICT OFFICE,e $5-10 million,500 or more,"2345 Grand Blvd., Suite 2400, KANSAS CITY, MO",NaN,NaN,NaN
498569,498569,498569,498569,499883,499883,9274667002,4/9/20,768,PPP,"TRIAD MANUFACTURING, INC",...,337 Furniture,3372 Office Furniture (including Fixtures) Man...,3372.0,ST. LOUIS DISTRICT OFFICE,e $5-10 million,250 to 499,"4321 Semple Avenue 0.0, St. Louis, MO",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514898,514898,514898,514898,516212,516212,9713238502,3/12/21,709,PPS,PREMIER PARKING SERVICES INC.,...,999 Other,8129 Other Personal Services,8129.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"104 E 5th St Ste 204, Kansas City, MO",NaN,NaN,NaN
514899,514899,514899,514899,516213,516213,9761878500,3/12/21,709,PPS,J DOOLE PROMOTIONS INC,...,323 Printing & Publishing,3231 Printing and Related Support Activities,3231.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6130 NW Bell Rd, Parkville, MO",NaN,NaN,NaN
514900,514900,514900,514900,516214,516214,9774858406,2/17/21,709,PPS,BUILDING CONTROL SOLUTIONS LLC,...,999 Other,2382 Building Equipment Contractors,2382.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1420 NW Vivion Rd Ste 103, Kansas City, MO",NaN,NaN,NaN
514901,514901,514901,514901,516215,516215,9851497109,4/15/20,709,PPP,"EPEC, INC.",...,999 Other,8132 Grantmaking and Giving Services,8132.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"5829 TROOST AVE, KANSAS CITY, MO",NaN,NaN,NaN


In [6]:
#process to identify and code for unique addresses 
cols = ["LoanNumber","full_add"]
ppp_togeoMO_add = ppp_togeoMO[cols]
ppp_togeoMO_add

,LoanNumber,full_add
498556,5467867110,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL..."
498557,5529817004,"520 W. Pennway St., Ste 300, KANSAS CITY, MO"
498558,6105917109,"2001 ProEnergy Blvd, SEDALIA, MO"
498566,8159007210,"2345 Grand Blvd., Suite 2400, KANSAS CITY, MO"
498569,9274667002,"4321 Semple Avenue 0.0, St. Louis, MO"
...,...,...
514898,9713238502,"104 E 5th St Ste 204, Kansas City, MO"
514899,9761878500,"6130 NW Bell Rd, Parkville, MO"
514900,9774858406,"1420 NW Vivion Rd Ste 103, Kansas City, MO"
514901,9851497109,"5829 TROOST AVE, KANSAS CITY, MO"


In [7]:
add = ppp_togeoMO["full_add"].astype(str).to_list()

In [8]:
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress?form'
driver.visit(url)

not_found = []
lat = []
long = []
geo_id = []

i=0
for i in tqdm(range(len(add))):
        driver.find_by_id('_address_id').clear()
        driver.find_by_id('_address_id').fill(add[i])
        driver.find_by_id('proceed').click()
        
        soup = BeautifulSoup(driver.html, 'html.parser')
        results = soup.find_all('div', id="pl_gov_census_geo_geocoder_domain_AddressResult")
        
        for result in results:
            result_text = result.text
            result_lat = re.findall('[^-](\d{2}[.]+\d+)', result_text)
            result_long = re.findall('[-]\d{2}[.]+\d+', result_text) #WEST 3-digit Longitude
            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('29')] #MO
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{add[i]} is {result_lat}, {result_long}, {fips_elements}")
                lat.append(result_lat[0])
                long.append(result_long[0])
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                not_found.append(i)
                lat.append(None)  
                long.append(None)
                geo_id.append(None)  
                print(f"{add[i]} is Not Found")
            except (TimeoutException):
                driver.navigate.refresh()
    
        sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_mac64.zip
Driver has been saved in cache [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/99.0.4844.51]


4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIELD, MO is Not Found
520 W. Pennway St., Ste 300, KANSAS CITY, MO is Not Found
2001 ProEnergy Blvd, SEDALIA, MO is Not Found
2345 Grand Blvd., Suite 2400, KANSAS CITY, MO is Not Found
4321 Semple Avenue 0.0, St. Louis, MO is Not Found
1878 S STATE HIGHWAY 125, ROGERSVILLE, MO is Not Found
101 HANLEY RD suite 300, SAINT LOUIS, MO is Not Found
391 St Jude Industrial Park 0, Marston, MO is Not Found
2411 WALTERS LN, PERRYVILLE, MO is Not Found
1 Athletic Drive, EARTH CITY, MO is Not Found
3500 Snyder Ave., SEDALIA, MO is Not Found
111 W Port Plz Ste 704, SAINT LOUIS, MO is Not Found
2345 Grand Blvd., Suite 2400, KANSAS CITY, MO is Not Found
722 Durham Rd, LEBANON, MO is Not Found
2100 NW Bypass, Springfield, MO is Not Found
190 Carondelet Plaza #1200, SAINT LOUIS, MO is Not Found
1100 Rockhurst Road, KANSAS CITY, MO is Not Found
2305 SOUTH HIGHWAY 65, MARSHALL, MO is Not Found
3933 BROADWAY  S, SAINT LOUIS, MO is Not Found
700 GRACE PKWY, FENTON

10301 N SERVICE RD, BOURBON, MO is Not Found
3612 Karnes Blvd, Ste 111, KANSAS CITY, MO is Not Found
TECHNOLOGY DR, LEES SUMMIT, MO is Not Found
906 Olive St Ste 700, Saint Louis, MO is Not Found
1300 NW BRIARCLIFF PKWY STE 275, KANSAS CITY, MO is Not Found
901 E. 114th Ave, KANSAS CITY, MO is Not Found
4001 Blue Parkway Suite 300, KANSAS CITY, MO is Not Found
200 trade center dr, SAINT PETERS, MO is Not Found
1950 Diamond Parkway, KANSAS CITY, MO is Not Found
301 N Memorial Dr, Saint Louis, MO is Not Found
242 HIGHWAY MM, LEBANON, MO is Not Found
PO Box 218, ALTON, MO is Not Found
201 Industrial Drive, Monett, MO is Not Found
1219 Highway 62 East, SIKESTON, MO is Not Found
2611 Schuetz Rd, MARYLAND HEIGHTS, MO is Not Found
3705 Business 61, PO Box 191, PALMYRA, MO is Not Found
120 NE SAINT LUKES BLVD Suite 200, LEES SUMMIT, MO is Not Found
22 North Euclid, Suite 220, SAINT LOUIS, MO is Not Found
11916 Missouri Bottom Rd, HAZELWOOD, MO is Not Found
3612 Karnes Blvd Ste 111, Kansas City

2900 NE Independent Ave, LEES SUMMIT, MO is Not Found
300 STATE ROUTE U, CARUTHERSVILLE, MO is Not Found
13573 ST. CHARLES ROAD ROAD, BRIDGETON, MO is Not Found
9237 WARD PKWY SUITE 300, KANSAS CITY, MO is Not Found
2 Eagle Industrial Dr, Troy, MO is Not Found
1000 CAMERA AVE Suite G, SAINT LOUIS, MO is Not Found
1000 Camera Ave Ste G, Saint Louis, MO is Not Found
P.O. Box 955, STRASBURG, MO is Not Found
62499 STATE HIGHWAY 15, NOVELTY, MO is Not Found
1000 Lake Saint Louis Blvd Ste 325, Lake St Louis, MO is Not Found
3009 N. BALLAS RD,Ste 100B, SAINT LOUIS, MO is Not Found
2790 CLAY EDWARDS DR STE 410, KANSAS CITY, MO is Not Found
1600 n 291 highway, INDEPENDENCE, MO is Not Found
1600 N 291 Highway Unit 390, Sugar Creek, MO is Not Found
1120 Intagliata Dr, Arnold, MO is Not Found
1120 Intagliata Drive, ARNOLD, MO is Not Found
3030 HADDOCK ST, WEST PLAINS, MO is Not Found
800 Newberry St, Fredericktown, MO is Not Found
1615 E 173rd St, Belton, MO is Not Found
26182 Creason Way, Carroll

2 Hawthorne Drive, HERMANN, MO is Not Found
1610 Des Peres Road, SAINT LOUIS, MO is Not Found
600 SARGENT DR, FREDERICKTOWN, MO is Not Found
1316 OLD HIGHWAY 63 STE 102, COLUMBIA, MO is Not Found
425 N. New Ballas Rd, Ste. 201, SAINT LOUIS, MO is Not Found
2535 Rose Parkway, Sikeston, MO is Not Found
5622 Delmar Blvd Suite 108, FLORISSANT, MO is Not Found
1000 Camera Ave Suite B, SAINT LOUIS, MO is Not Found
808 Highway 34 W, MARBLE HILL, MO is Not Found
BROADWAY ST, CAPE GIRARDEAU, MO is Not Found
301 AUTUMN RIDGE DR, HERCULANEUM, MO is Not Found
230 Carondelet Commons Blvd River City Business Park 1, Saint Louis, MO is Not Found
2 E STE MARIES, PERRYVILLE, MO is Not Found
111 WESTPORT PLZ STE 500, SAINT LOUIS, MO is Not Found
808 Hunter Ave #1A, SIKESTON, MO is Not Found
13560 BARRETT PARKWAY DR, SAINT LOUIS, MO is Not Found
2101 Corona Rd Suite 102, COLUMBIA, MO is Not Found
4401 I-70 Drive SE, COLUMBIA, MO is Not Found
2274 South Business HWY 71, ANDERSON, MO is Not Found
2650 Coun

...

727 CEDAR CT, TWN AND CNTRY, MO is Not Found
811 Grand Blvd Ste 101, KANSAS CITY, MO is Not Found
503 Nifong Blvd #215, COLUMBIA, MO is Not Found
206 NW Mock Ave Ste 100, Blue Springs, MO is Not Found
308 NOONAN DR, PACIFIC, MO is Not Found
9660 N SEYMOUR, KANSAS CITY, MO is Not Found
11675 Lilburn Park Rd, Saint Louis, MO is Not Found
11675 Lilburn Park Rd, SAINT LOUIS, MO is Not Found
4120 RIDER TRL, EARTH CITY, MO is Not Found
5142 Osage Beach Pkwy, Osage Beach, MO is Not Found
13537 Barrett Parkway Dr Ste 110, Ballwin, MO is Not Found
1123 WILKES BLVD suite 250, COLUMBIA, MO is Not Found
.3940 MAIN ST, GRANDVIEW, MO is Not Found
12935 GRAVOIS RD, SAINT LOUIS, MO is Not Found
2405 GRAND BLVD SUITE 800, KANSAS CITY, MO is Not Found
14788 Business 13, REEDS SPRING, MO is Not Found
P.O. Box 901502, Kansas City, MO is Not Found
401 P LANE, ALTON, MO is Not Found
3016 Highway T, Foristell, MO is Not Found
808 Springfield, Rd, AVA, MO is Not Found
410 STERLING AVE, KANSAS CITY, MO is Not 

16150 MAIN CIRCLE DR, STE 250, CHESTERFIELD, MO is Not Found
2339 US Highway 65, WALNUT SHADE, MO is Not Found
122 LIEBER ST E, MARYVILLE, MO is Not Found
208 N Business Hwy 61, Bowling Green, MO is Not Found
8137 Sante Fe Drive, LEES SUMMIT, MO is Not Found
9501 N Oak Trfy Ste 205, Kansas City, MO is Not Found
5282 S 118TH RD, MORRISVILLE, MO is Not Found
PO Box 416, WILLARD, MO is Not Found
1545 N. U.S. Highway 67, FLORISSANT, MO is Not Found
8301 MARYLAND AVE Ste 100, SAINT LOUIS, MO is Not Found
456 N New Ballas Rd Ste 129, Saint Louis, MO is Not Found
930 Khers Mill Rd. #314, BALLWIN, MO is Not Found
1910 SAINT LOUIS ST, SPRINGFIELD, MO is Not Found
#31 The Blvd, SAINT LOUIS, MO is Not Found
815 OLIVE ST Ste 110, SAINT LOUIS, MO is Not Found
13023 TESSON FERRY RD Ste 105, SAINT LOUIS, MO is Not Found
7778 Winghaven Blvd, O FALLON, MO is Not Found
11605 STUDT AVE STE 105, SAINT LOUIS, MO is Not Found
3009 N BALLAS RD STE 371C, SAINT LOUIS, MO is Not Found
PO Box 14440, SPRINGFIELD,

1601 Chapel Hill Rd Ste A, Columbia, MO is Not Found
P O Box 681430, Riverside, MO is Not Found
One Royal Way, KANSAS CITY, MO is Not Found
1710 GIBSON AVE, WEST PLAINS, MO is Not Found
2056 Highway AA, Poplar Bluff, MO is Not Found
2801 WOODARD DR Suite 103, COLUMBIA, MO is Not Found
1021 NE JIB Ct STE A - B, Lee's Summit, MO is Not Found
10929 INDUSTRIAL DR, SAINTE GENEVIEVE, MO is Not Found
17001 S State Route 291, Pleasant Hill, MO is Not Found
17001 S. STATE ROUTE 291, PLEASANT HILL, MO is Not Found
1507 Hwy Z, WENTZVILLE, MO is Not Found
17300 North Outer 40 Rd Ste 300, Chesterfield, MO is Not Found
6967 TUCKER, ODESSA, MO is Not Found
3804 N Belt Hwy, Saint Joseph, MO is Not Found
16150 Main Circle Dr, Ste. 205, CHESTERFIELD, MO is Not Found
3449 AMERICAN AVE STE B, JEFFERSON CITY, MO is Not Found
2352 HIGHWAY K, O FALLON, MO is Not Found
710 North 2ND Suite 300S St, SAINT LOUIS, MO is Not Found
14601 Allison Industrial Dr PO BOX 80, HALLSVILLE, MO is Not Found
2601 Oak Ridge Ex

1615 BROADWAY  S, SAINT LOUIS, MO is Not Found
2211 W MAIN ST, SEDALIA, MO is Not Found
13014 BUTLER CREST DR, SAINT LOUIS, MO is Not Found
6189 HIGHWAY Y, GERALD, MO is Not Found
16506 E US Highway 40, Independence, MO is Not Found
1 HC 64 Box 4590, MARBLE HILL, MO is Not Found
991 STATE HIGHWAY EE, CHAFFEE, MO is Not Found
4173 20TH ST N, OZARK, MO is Not Found
1610 DES PERES RD STE 150, ST LOUIS, MO is Not Found
2502 Tanner Dr Ste 101, Cape Girardeau, MO is Not Found
4240 Duncan Ave Suite 200, SAINT LOUIS, MO is Not Found
3320 S OUTER BELT RD, GRAIN VALLEY, MO is Not Found
4215 PHILLIPS FARM ROAD SUITE 101A, COLUMBIA, MO is Not Found
12775 NEWPORT PL, LA PLATA, MO is Not Found
7827 Town Square Ave Ste 104, O Fallon, MO is Not Found
220 W Lockwood, Ste 203, SAINT LOUIS, MO is Not Found
2 The Pines Ct, Saint Louis, MO is Not Found
1818 AVENUE, SAINT LOUIS, MO is Not Found
1113 Private Road 1006, CLIFTON HILL, MO is Not Found
322A Emerson Rd, High Ridge, MO is Not Found
330 State Highw

1420 NW Vivion Rd Ste 103, Kansas City, MO is Not Found



In [9]:
print(len(add))
print(len(lat))
print(len(long))
print(len(geo_id))

6946
6947
6946
6946


<b>Add remaining values (2801) to result_df(5461)

In [ ]:
pd.set_option("display.max_rows", None)
lat_df = pd.DataFrame({"Lat":lat})
lat_df_drop = lat_df.drop(index=[958],axis=1)
lat_list = lat_df_drop["Lat"].to_list()
lat_list

In [ ]:
result_df = pd.DataFrame({"full_add":add, "Lat":lat_list, "Long":long, "FIPS_z": geo_id})
result_df 

In [ ]:
result_df['FIPS_z'] = result_df['FIPS_z'].str[0]
result_df 

In [22]:
#mapping coordinates to NaN values in main 
m = dict(zip(result_df.full_add,result_df.Lat))
ppp_togeoMO.Lat = ppp_togeoMO.Lat.fillna(ppp_togeoMO.full_add.map(m))
n = dict(zip(result_df.full_add,result_df.Long))
ppp_togeoMO.Long = ppp_togeoMO.Long.fillna(ppp_togeoMO.full_add.map(n))
o = dict(zip(result_df.full_add,result_df.FIPS_z))
ppp_togeoMO.FIPS_z = ppp_togeoMO.FIPS_z.fillna(ppp_togeoMO.full_add.map(o))

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
ppp_togeoMO

In [24]:
cols = ["LoanNumber", "FIPS_z", "Lat", "Long"]
ppp_togeoMO_parsed = ppp_togeoMO[cols]
ppp_togeoMO_parsed.head()

,LoanNumber,FIPS_z,Lat,Long
498556,5467867110,None,None,None
498557,5529817004,None,None,None
498558,6105917109,None,None,None
498566,8159007210,None,None,None
498569,9274667002,None,None,None


In [25]:
ppp_geo = ppp_geo.merge(ppp_togeoMO_parsed, on="LoanNumber", how="left")
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat_x,Long_x,FIPS_z_x,FIPS_z_y,Lat_y,Long_y
0,0,0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,NaN,NaN,NaN,NaN
1,1,1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,NaN,NaN,NaN,NaN
2,2,2,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,...,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,NaN,NaN,NaN,NaN
3,3,3,3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,...,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,NaN,NaN,NaN,NaN
4,4,4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,...,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,NaN,NaN,NaN,NaN


In [26]:
ppp_geo["Lat"] = ppp_geo["Lat_y"].fillna(ppp_geo["Lat_x"])
ppp_geo["Long"] = ppp_geo["Long_y"].fillna(ppp_geo["Long_x"])
ppp_geo["FIPS_z"] = ppp_geo["FIPS_z_y"].fillna(ppp_geo["FIPS_z_x"])
ppp_geo = ppp_geo.drop(["Lat_x", "Long_x", "Lat_y", "Long_y", "FIPS_z_y", "FIPS_z_x"], axis=1)
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.8537,-80.3699,NaN
1,1,1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.9423,-80.0662,NaN
2,2,2,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.6361,-122.303,NaN
3,3,3,3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,...,999 Other,2361 Residential Building Construction,2361.0,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.2247,-117.373,NaN
4,4,4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.5218,-82.9137,NaN


In [27]:
ppp_geo = ppp_geo.drop(["Unnamed: 0", "Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1", "Unnamed: 0.1.1.1.1"], axis=1)
ppp_geo.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,SC,29150.0,12/18/20,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.8537,-80.3699,NaN
1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,SC,29420.0,9/28/21,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.9423,-80.0662,NaN
2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,WA,98112.0,3/17/21,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.6361,-122.303,NaN
3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,CA,92407.0,10/16/21,...,999 Other,2361 Residential Building Construction,2361.0,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.2247,-117.373,NaN
4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,Roseville,MI,NaN,8/17/21,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.5218,-82.9137,NaN


In [28]:
ppp_geo.to_csv("data/state_data/geo/ppp_geo.csv")

In [ ]:
ppp_geo_mo = ppp_geo.loc[ppp_geo["BorrowerState"] =="MO"]
ppp_geo_mo

In [ ]:
null_check = ppp_geo_mo.loc[ppp_geo_mo["Lat"].isnull()]
null_check